In [2]:
%matplotlib inline
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')

In [3]:
df = pd.read_csv('data/KBOS-2012.csv')

In [5]:
df.columns

Index(['EST', 'Max TemperatureF', 'Mean TemperatureF', 'Min TemperatureF',
       'Max Dew PointF', 'MeanDew PointF', 'Min DewpointF', 'Max Humidity',
       ' Mean Humidity', ' Min Humidity', ' Max Sea Level PressureIn',
       ' Mean Sea Level PressureIn', ' Min Sea Level PressureIn',
       ' Max VisibilityMiles', ' Mean VisibilityMiles', ' Min VisibilityMiles',
       ' Max Wind SpeedMPH', ' Mean Wind SpeedMPH', ' Max Gust SpeedMPH',
       'PrecipitationIn', ' CloudCover', ' Events', ' WindDirDegrees'],
      dtype='object')

### Preprocessing the data
We quantize the measured precipitation in inch into four categories: 1="dry or very light rain", 2="light rain", 3="moderate rain", 4="heavy rain". Sometimes, the precipitation could not be recorded accurately and the data show "T" for trace. We set these to 0.

In [19]:
precipitation = df['PrecipitationIn'].values
precipitation[precipitation == 'T'] = 0.
precipitation = precipitation.astype(float)

In [29]:
# we quantize using the following regions (acc. to wiki):
# 1: [0.0, 0.098)
# 2: [0.098, 0.3)
# 3: [0.3, 2.)
# 4: [2., +inf)

q_precipitation = precipitation.copy()
q_precipitation[precipitation < 0.098] = 1
q_precipitation[(precipitation >= 0.098) * (precipitation < 0.3)] = 2
q_precipitation[(precipitation >= 0.3) * (precipitation < 2.)] = 3
q_precipitation[precipitation >= 2.] = 4